In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='darkgrid', context='talk', palette='Dark2')

In [75]:
data = pd.read_csv('C:\\Users\\p998nkf\\Downloads\\SWED-A.ST.csv', index_col = 'Date')

data = data.where(data['Close'] != 'null')
data = data.astype('float64')
data.index = pd.to_datetime(data.index)

print(data[['Open', 'Close']].describe())

data['50D SMA'] = data['Close'].rolling(50).mean()
data['200D SMA'] = data['Close'].rolling(200).mean()
data['50D STD'] = data['Close'].rolling(50).std()
data['200D STD'] = data['Close'].rolling(200).std()
data['200D VOL'] = data['Volume'].rolling(200).mean()
data = data.dropna()

vol = data['Volume']
data = data.drop('Volume', axis=1)

data = data.pct_change().dropna()
data['Volume'] = vol[-len(data.index):]

              Open        Close
count  4752.000000  4752.000000
mean    137.420371   137.405966
std      50.030071    50.024129
min      16.460501    15.219800
25%      98.432098    98.432098
50%     128.623001   128.623001
75%     181.112003   181.000000
max     248.500000   248.500000


In [76]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3291 entries, 2000-10-09 to 2018-11-02
Data columns (total 11 columns):
Open         3291 non-null float64
High         3291 non-null float64
Low          3291 non-null float64
Close        3291 non-null float64
Adj Close    3291 non-null float64
50D SMA      3291 non-null float64
200D SMA     3291 non-null float64
50D STD      3291 non-null float64
200D STD     3291 non-null float64
200D VOL     3291 non-null float64
Volume       3291 non-null float64
dtypes: float64(11)
memory usage: 308.5 KB


In [77]:
y = data['Close'][1:]
x = data.drop('Close', axis=1)[:-1]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.13, random_state=42, shuffle = False)
i = data.index[-1]

print(x.tail(2))
print(y.tail(2))

print('\nTrain samples: {}\nTest samples: {}'.format(len(y_train), len(y_test)))

                Open      High       Low  Adj Close   50D SMA  200D SMA  \
Date                                                                      
2018-10-31  0.008321  0.009732  0.014321   0.002920 -0.000289  0.000143   
2018-11-01  0.000485 -0.003373 -0.008277  -0.005822 -0.000299  0.000113   

             50D STD  200D STD  200D VOL     Volume  
Date                                                 
2018-10-31  0.000060  0.001189 -0.001503  2889150.0  
2018-11-01  0.001067  0.000799 -0.000903  2600307.0  
Date
2018-11-01   -0.005822
2018-11-02    0.000000
Name: Close, dtype: float64

Train samples: 2862
Test samples: 428


**Random Forest Regression**

In [78]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

#-1 for n_jobs means using all processors for the fit/predict
modRF = RandomForestRegressor(n_estimators = 200, n_jobs = -1)
randomF = modRF.fit(X_train, y_train)

print('Test score: {}'.format(round(randomF.score(X_test, y_test),6)))

Test score: -0.376353


In [79]:
results = pd.DataFrame()
#results['ACTUAL'] = y_test
actual = []
test = []

if len(y_test) > 252:
    n = 252
else:
    n = len(y_test)

for i in range(0, n-1):
    t = x.iloc[-n + i:].values
    #.reshape(1, -1)
    actual.append(y.iloc[-n + i])
    randomF = modRF.fit(x[:-n + i], y[:-n + i])
    test.append(randomF.predict(t)[0])
    
results['ACTUAL'] = actual
results['RANDOM FOREST'] = test
results['RF ACC'] = round((results['RANDOM FOREST'] - results['ACTUAL'])/results['ACTUAL'], 2)

results[['ACTUAL', 'RANDOM FOREST']].plot()
plt.show()

results.mean()

KeyboardInterrupt: 

In [83]:
print(max(x.index), max(data.index))
randomF = modRF.fit(x, y)
randomF.predict(data.drop('Close', axis=1).iloc[-1].reshape(1, -1))

2018-11-01 00:00:00 2018-11-02 00:00:00


C:\Users\p998nkf\AppData\Local\conda\conda\envs\another_root\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until


array([-0.00201073])

In [47]:
import statsmodels.api as sm
mResult = []
actual = []
m = int(len(y.values)/3)

for i in range(0, m):
    try:
        mod_MarkovRegres = sm.tsa.MarkovAutoregression(y[-(m-i+1):], k_regimes=2,\
                                                       order = 2,\
                                                       switching_ar = True,\
                                                       switching_trend = True,\
                                                       switching_variance = True)
        MarkovRegres = mod_MarkovRegres.fit()
        
        mResult.append(MarkovRegres.predict(start=0, end=1))
        actual.append(y.values[-(m-i+1)])
        
    except ValueError:
        break
        
    finally:
        if m-i+1<20: mm = m-i+1

MarkovRegres.summary()

C:\Users\p998nkf\AppData\Local\conda\conda\envs\another_root\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\p998nkf\AppData\Local\conda\conda\envs\another_root\lib\site-packages\statsmodels\tsa\regime_switching\markov_switching.py:391: RuntimeWarning: invalid value encountered in true_divide
  predicted_joint_probabilities[..., t+1])
C:\Users\p998nkf\AppData\Local\conda\conda\envs\another_root\lib\site-packages\statsmodels\tsa\regime_switching\markov_autoregression.py:231: RuntimeWarning: overflow encountered in exp
  np.exp(-0.5 * resid**2 / variance) / np.sqrt(2 * np.pi * variance))
C:\Users\p998nkf\AppData\Local\conda\conda\envs\another_root\lib\site-packages\statsmodels\tsa\regime_switching\markov_autoregression.py:231: RuntimeWarning: invalid value encountered in true_divide
  np.exp(-0.5 * resid**2 / variance) / np.sqrt(2 * np.pi * varian

<class 'statsmodels.iolib.summary.Summary'>
"""
                         Markov Switching Model Results                         
================================================================================
Dep. Variable:                    Close   No. Observations:                    1
Model:             MarkovAutoregression   Log Likelihood                     nan
Date:                  Mon, 05 Nov 2018   AIC                                nan
Time:                          14:50:35   BIC                                nan
Sample:                      10-31-2018   HQIC                               nan
                           - 10-31-2018                                         
Covariance Type:                 approx                                         
                             Regime 0 parameters                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const             nan        nan        nan        nan         nan         nan
sigma2            nan        nan        nan        nan         nan         nan
ar.L1             nan        nan        nan        nan         nan         nan
ar.L2             nan        nan        nan        nan         nan         nan
                             Regime 1 parameters                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const             nan        nan        nan        nan         nan         nan
sigma2            nan        nan        nan        nan         nan         nan
ar.L1             nan        nan        nan        nan         nan         nan
ar.L2             nan        nan        nan        nan         nan         nan
                         Regime transition parameters                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p[0->0]           nan        nan        nan        nan         nan         nan
p[1->0]           nan        nan        nan        nan         nan         nan
==============================================================================

Warnings:
[1] Covariance matrix is singular or near-singular, with condition number    nan. Standard errors may be unstable.
"""

In [48]:
res = []

for p in range(0, len(mResult)):
    
    if mResult[p][1] > 0 and actual[p] > 0:
        res.append(1)
    elif mResult[p][1] < 0 and actual[p] < 0:
        res.append(1)
    else:
        res.append(0)
    
np.mean(res)

0.0

***The End***

In [49]:
results.to_csv('results.csv', encoding = 'utf-16')